# Import

In [1]:
from tqdm.notebook import tqdm

import mlflow

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

# Load Dataset

In [2]:
train_set = datasets.MNIST(
    "mnist_data",
    train=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,),(0.3081,))
    ]),
    download=True
)

test_set = datasets.MNIST(
    "mnist_data",
    train=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,),(0.3081,))
    ]),
    download=True
)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:02<00:00, 4.73MB/s]


Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 152kB/s]


Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.41MB/s]


Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 2.02MB/s]

Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



# Define Model

In [4]:
class MNISTClassifier(nn.Module):
    def __init__(self, num_classes, hidden_channels, kernel_size, dropout):
        super(MNISTClassifier, self).__init__()
        
        self.conv_1 = nn.Conv2d(1, hidden_channels[0], kernel_size=kernel_size)

        self.conv_2 = nn.Conv2d(hidden_channels[0], 20, kernel_size=kernel_size)

        self.conv_2_drop = nn.Dropout2d(p=dropout)

        self.maxpool = F.max_pool2d()

        self.fc1 = nn.Linear(320, 50)
        self.dropout = F.dropout()

        self.fc2 = nn.Linear(50, num_classes)

        self.activation = F.relu()

        self.log_softmax = F.log_softmax()

    def forward(self, x):
        x = self.conv_1(x)
        x = self.maxpool(x, 2)
        x = self.activation(x)

        x = self.conv_2(x)
        x = self.conv_2_drop(x)
        x = self.maxpool(x, 2)
        x = self.activation(x)

        x = x.view(-1, 320)

        x = self.fc1(x)
        x = self.activation(x)

        x = self.dropout(x, training=self.training)
        x = self.fc2(x)
        
        output = self.log_softmax(x, dim=1)

        return output

# Define Performance Logging Function

In [5]:
def log_performance(model, data_loader, device, epoch, metric_type="Test"):
    model.eval()

    loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in tqdm(data_loader, total=len(data_loader), desc=f"Log {metric_type} Perfomance >"):
            data, target = data.to(device), target.to(device)

            output = model(data)

            loss += F.nll_loss(
                output, target,
                reduction="sum"
            ).item()

            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(target.view_as(prediction)).sum().item()

        loss /= len(data_loader.dataset)
        accuracy = 100.0 * correct / len(data_loader.dataset)
    
    mlflow.log_metric(f"{metric_type}_loss", loss, step=epoch)
    mlflow.log_metric(f"{metric_type}_accuracy", accuracy, step=epoch)

# Define Training Function

In [ ]:
def train_model(
        train_set,
        test_set,
        param_set,
        experiment_name,
        run_name,
        optimizer="sgd"
):
    mlflow.set_experiment(experiment_name=experiment_name)

    with mlflow.start_run(run_name=run_name):
        mlflow.pytorch.autolog()

        mlflow.log_param('batch_size', param_set['batch_size'])
        mlflow.log_param('hidden_channels', param_set['hidden_chennels'])
        mlflow.log_param('kernel_size', param_set['kernel_size'])
        mlflow.log_param('dropout', param_set['dropout'])
        mlflow.log_param('momentum', param_set['momentum'])
        mlflow.log_param('learning_rate', param_set['learning_rate'])
        mlflow.log_param('epochs', param_set['epochs'])
        mlflow.log_param('optimizer', optimizer)

        device = torch.device("cpu")

        torch.manual_seed(42)

        train_loader = DataLoader(train_set, batch_size=param_set['batch_size'], shuffle=True)
        test_loader = DataLoader(test_set, batch_size=1000, shuffle=True)

        model = MNISTClassifier(
            num_classes=10,
            
        )